# HEAL DATA SYNC - Jupyter Notebook Version

In [ ]:

# Install necessary dependencies (if not installed)
!pip install openai langchain qdrant-client fastapi uvicorn langsmith ragas chainlit


In [ ]:

import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.graphs import Graph
from langsmith import LangSmith
from ragas import Ragas


In [ ]:

# Set API keys and environment variables
OPENAI_API_KEY = "your_openai_api_key"
QDRANT_URL = "http://localhost:6333"


In [ ]:

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)

# Initialize Embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY)


In [ ]:

# Initialize Qdrant Vector Database
vector_db = Qdrant(embedding_function=embeddings, url=QDRANT_URL)


In [ ]:

# Initialize LangGraph
graph = Graph()


In [ ]:

def data_pipeline(query: str):
    """Handles end-to-end data retrieval and processing."""
    embedding = embeddings.embed_query(query)
    results = vector_db.similarity_search_by_vector(embedding)
    response = llm.invoke(" ".join([doc.page_content for doc in results]))
    return response


In [ ]:

# Test LLM response
query_text = "Explain the impact of AI on healthcare."
response = data_pipeline(query_text)
print("Response:", response)


In [ ]:

# Initialize Monitoring
monitoring = LangSmith(project_name="HEAL_DATA_SYNC")
monitoring.track_pipeline(data_pipeline)


In [ ]:

# Initialize Evaluation
evaluation = Ragas()

def evaluate_response(response: str, query: str):
    """Evaluate response quality using RAGAS."""
    return evaluation.score_response(query, response)


In [ ]:

# Test Evaluation
score = evaluate_response(response, query_text)
print("Evaluation Score:", score)
